In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


IMPORTING ALL THE NECESSARY LIBRARIES:

In [1]:
import numpy as np
import pandas as pd
import os
import librosa
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,TimeDistributed
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

from keras.utils import np_utils
from sklearn import metrics
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

In [ ]:
!unzip /content/drive/MyDrive/validation.zip -d /content/drive/MyDrive  #UNZIP THE TRAIN DATASET

In [3]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

PRE PROCESSING DATA

In [6]:
i=0
f_input={}
for name in os.listdir("/content/drive/MyDrive/X"):
    s="/content/drive/MyDrive/X/"+name
    t=np.load(s)
    size=len(name)
    data=0
    j=0
    while(name[j]!="_"):
        j=j+1
    j=j+1
    while(j<size-4):
        data=data*10+int(ord(name[j])-ord("0"))
        j=j+1
    f_input[data]=t[0]
    i=i+1
    

In [ ]:
print(f_input)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
i=0
f_output={}
arr1_y={}
for name in os.listdir("/content/drive/MyDrive/Y"):
    s="/content/drive/MyDrive/Y/"+name
    t=np.load(s)
    size=len(name)
    data=0
    j=0
    while(name[j]!='_'):
        j=j+1
    j=j+1
    while(j<size-4):
        data=data*10+int(ord(name[j])-ord('0'))
        j=j+1
    f_output[data]=eventroll_to_multihot_vector(t)
    i=i+1

In [8]:
print(f_output)

{91: array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]), 9099: array([0., 0., 0., 0., 0., 0., 1., 0., 1., 0.]), 910: array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), 9100: array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]), 9102: array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]), 9101: array([0., 0., 0., 1., 0., 0., 0., 1., 1., 0.]), 9104: array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]), 9103: array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0.]), 9107: array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]), 9105: array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]), 9106: array([1., 1., 1., 0., 1., 0., 0., 1., 1., 0.]), 9109: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 9108: array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0.]), 911: array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0.]), 9110: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 9112: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 9111: array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]), 9113: array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]), 9114: array([

In [9]:
final=[]
for i in range(0,10000):
    final.append([f_input[i],f_output[i]])   #appending input and output

In [10]:
dataframe=pd.DataFrame(final,columns=['feature','class'])

In [11]:
dataframe.head()

,feature,class
0,"[[0.1753835, 0.18565877, 0.18828404, 0.1854411...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
1,"[[0.1120325, 0.12441835, 0.13066268, 0.1346521...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
2,"[[0.19059084, 0.20715885, 0.21303593, 0.203357...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,"[[0.2193706, 0.21709621, 0.20100619, 0.1762448...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
4,"[[0.27521786, 0.29266554, 0.30288097, 0.301031...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [12]:
dataframe.iloc[2,0].shape

(64, 1000)

In [13]:
x=np.array(dataframe['feature'][0:10000].to_list())
y=np.array(dataframe['class'][0:10000].to_list())

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [15]:
x_train=x_train.reshape(8000,64,1000,1)
x_test=x_test.reshape(2000,64,1000,1)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

In [16]:
print(x_train.shape)

(8000, 64, 1000, 1)


In [65]:
import keras
import tensorflow as tf
from keras import regularizers, activations
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import layers

model = Sequential(
    [
        Conv2D(filters=16, kernel_size = (4, 8), strides=(2, 4), activation='relu', input_shape = x_train[0].shape), 
        MaxPooling2D(pool_size = (2,4)),
        Dropout(0.2),
     
        Conv2D(filters=32, kernel_size=(2, 4), strides=(1, 2), activation='relu'), 
        MaxPooling2D(pool_size = (2,2)),
        Dropout(0.2),
     
        Conv2D(filters=64, kernel_size=(2,2), activation='relu'), 
        MaxPooling2D(pool_size = (2,2)),
        Dropout(0.2),
     

        Flatten(),

      
        Dense(300,activation='relu'),
        Dropout(0.2),
        Dense(300,activation='relu'),
        Dropout(0.2),
        Dense(10,activation='sigmoid')
               ])
model.summary()

 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 31, 249, 16)       528       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 62, 16)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 15, 62, 16)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 30, 32)        4128      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 15, 32)        0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 7, 15, 32)        

In [19]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [66]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',metrics=[keras.metrics.BinaryAccuracy()])



In [67]:
history = model.fit(x_train, y_train, epochs=40, batch_size=32,validation_data=(x_test,y_test))

Epoch 1/40
250/250 [==============================] - 36s 139ms/step - loss: 0.3727 - binary_accuracy: 0.8677 - val_loss: 0.3429 - val_binary_accuracy: 0.8697
Epoch 2/40
250/250 [==============================] - 42s 169ms/step - loss: 0.3329 - binary_accuracy: 0.8703 - val_loss: 0.3220 - val_binary_accuracy: 0.8724
Epoch 3/40
250/250 [==============================] - 35s 140ms/step - loss: 0.3190 - binary_accuracy: 0.8733 - val_loss: 0.3127 - val_binary_accuracy: 0.8747
Epoch 4/40
250/250 [==============================] - 34s 135ms/step - loss: 0.3113 - binary_accuracy: 0.8747 - val_loss: 0.3115 - val_binary_accuracy: 0.8745
Epoch 5/40
250/250 [==============================] - 32s 129ms/step - loss: 0.3021 - binary_accuracy: 0.8777 - val_loss: 0.2949 - val_binary_accuracy: 0.8808
Epoch 6/40
250/250 [==============================] - 33s 132ms/step - loss: 0.2924 - binary_accuracy: 0.8825 - val_loss: 0.2858 - val_binary_accuracy: 0.8839
Epoch 7/40
250/250 [==========================

In [68]:
model.save("Sriram_ee603_A2.h5")  #SAVE THE MODEL SO THAT IF SYSTEM CRASHES, MODEL CAN BE RETRIEVED

In [ ]:
#import keras
#model = keras.models.load_model("Sriram_ee603_A2.h5")

PREPROCESSING THE VALIDATION DATASET

In [69]:
i=0
f_val_input={}
lst=[]
for name in os.listdir("/content/drive/MyDrive/dataset/X"):
    s="/content/drive/MyDrive/dataset/X/"+name
    t=np.load(s)
    size=len(name)
    data=0
    j=0
    while(name[j]!="_"):
        j=j+1
    j=j+1
    while(j<size-4):
        data=data*10+int(ord(name[j])-ord("0"))
        j=j+1
    f_val_input[data]=t[0]
    lst.append(data)
    i=i+1
    

In [70]:
print(len(f_val_input))

2000


In [72]:
i=0
f_val_output={}
arr1_y={}
for name in os.listdir("/content/drive/MyDrive/dataset/Y"):
    s="/content/drive/MyDrive/dataset/Y/"+name
    t=np.load(s)
    size=len(name)
    data=0
    j=0
    while(name[j]!='_'):
        j=j+1
    j=j+1
    while(j<size-4):
        data=data*10+int(ord(name[j])-ord('0'))
        j=j+1
    f_val_output[data]=eventroll_to_multihot_vector(t)
    i=i+1

In [73]:
print(f_val_output)

{2026: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 966: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 1693: array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), 2661: array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0.]), 252: array([1., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), 3879: array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]), 3590: array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), 929: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 1193: array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 2733: array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), 1139: array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 1370: array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0.]), 3514: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 1420: array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 36: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 1867: array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), 388: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 2530: array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0.]), 2173: array([0.

In [74]:
final_val=[]
for i in lst:
    final_val.append(f_val_input[i])

In [75]:
y_val=[]
for i in lst:
    y_val.append(f_val_output[i])

In [76]:
final_val = np.array(final_val) #converting the list to array

In [77]:
y_prediction = model.predict(final_val) #PREDICTION OF VALIDATION DATASET

63/63 [==============================] - 2s 36ms/step


In [78]:
print(y_prediction.shape)

(2000, 10)


In [79]:
y_prediction = np.array(y_prediction) 

In [80]:
y_prediction=y_prediction.round() #rounding off the decimal values to integers

In [84]:
from sklearn.metrics import precision_score
print(precision_score(y_prediction,y_val,average="samples"))    #calculating precision score

0.6132833333333333


In [85]:
!unzip /content/drive/MyDrive/test.zip  #UNZIP THE TEST DATASET

Archive:  /content/drive/MyDrive/test.zip
   creating: test/
   creating: test/X/
  inflating: test/X/melspec_1014.npy  
  inflating: test/X/melspec_996.npy  
  inflating: test/X/melspec_2082.npy  
  inflating: test/X/melspec_406.npy  
  inflating: test/X/melspec_538.npy  
  inflating: test/X/melspec_611.npy  
  inflating: test/X/melspec_312.npy  
  inflating: test/X/melspec_1167.npy  
  inflating: test/X/melspec_421.npy  
  inflating: test/X/melspec_1149.npy  
  inflating: test/X/melspec_1626.npy  
  inflating: test/X/melspec_1696.npy  
  inflating: test/X/melspec_695.npy  
  inflating: test/X/melspec_1718.npy  
  inflating: test/X/melspec_1436.npy  
  inflating: test/X/melspec_2057.npy  
  inflating: test/X/melspec_1009.npy  
  inflating: test/X/melspec_732.npy  
  inflating: test/X/melspec_886.npy  
  inflating: test/X/melspec_706.npy  
  inflating: test/X/melspec_419.npy  
  inflating: test/X/melspec_894.npy  
  inflating: test/X/melspec_386.npy  
  inflating: test/X/melspec_606.np

PREPROCESSING TEST DATA

In [112]:
i=0
f_test_input={}
filename=[]
lst=[]
for name in os.listdir("/content/test/X/"):
    s="/content/test/X/"+name
    t=np.load(s)
    size=len(name)
    data=0
    j=0
    while(name[j]!="_"):
        j=j+1
    j=j+1
    while(j<size-4):
        data=data*10+int(ord(name[j])-ord("0"))
        j=j+1
    f_test_input[data]=t[0]
    lst.append(data)
    filename.append(name)
    i=i+1

In [113]:
print(len(f_test_input))

2500


In [ ]:
print(f_test_input)

In [115]:
final_test=[]
for i in lst:
    final_test.append(f_test_input[i])

In [116]:
final_test = np.array(final_test)

In [117]:
y_test = model.predict(final_test)  #FINAL PREDICTION

79/79 [==============================] - 3s 32ms/step


In [118]:
y_test=y_test.round()  #ROUNDING OFF THE PROBABILITY TO INTEGER

CONVERTING THE PREDICTED VALUES TO ACTUAL SOUND

In [121]:
PREDICTION=[]
annotate = ['Alarm_bell_ringing','Blender','Cat','Dishes','Dog','Electric_shaver_toothbrush','Frying','Running_water','Speech','Vacuum_cleaner']
for j in y_test:
    one = []
    for i in range(0,10):
        if j[i]==1:
            one.append(annotate[i])
    if(len(one)==0):
        one.append('silence')
    one = np.array(one)
    PREDICTION.append(one)

In [122]:
for sound in PREDICTION:   #FINAL PREDICTIONS
  print(sound)

['Dishes' 'Frying' 'Speech']
['Speech' 'Vacuum_cleaner']
['Speech' 'Vacuum_cleaner']
['Dishes' 'Frying' 'Speech']
['Running_water' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Running_water' 'Speech']
['Dishes' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Cat' 'Speech']
['Dishes' 'Frying' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Dishes' 'Frying' 'Speech']
['Dog' 'Speech']
['Blender' 'Speech']
['Blender' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Speech']
['Alarm_bell_ringing' 'Dog' 'Speech']
['Dishes' 'Frying' 'Speech']
['Speech']
['Dishes' 'Frying' 'Speech']
['Electric_shaver_toothbrush' 'Speech']
['Dishes' 'Frying' 'Speech']
['Running_water' 'Speech']
['Speech' 'Vacuum_cleaner']
['Dog' 'Speech']
['Blender' 'Speech' 'Vacuum_cleaner']
['Speech']
['Alarm_bell_ringing' 'Speech']
['Dog' 'Speech']
['Dog' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Speech' 'Vacuum_cleaner']
['Cat' 'Speech']
['Dog' 'Speech']
['Alarm_bell_ringing' 'Speech']
['Speech']
['Dishes' 'Frying' 'Speech']
['Running_

CREATING CSV FILE

In [123]:

import pandas as pd    
df = pd.DataFrame({"filename":filename,"preds":PREDICTION})
df.to_csv("200744_A2_EE603.csv",index=False)   